# 0. Информация об видеокарте

In [8]:
!nvidia-smi

Sat Apr 13 16:55:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   38C    P8               7W /  30W |    512MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1. Загрузка библиотек и объявление путей

In [1]:
import os
import glob
import random
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import json
from pprint import pprint
import shutil
import yaml
from ultralytics import YOLO

import warnings
warnings.filterwarnings("ignore")

/home/k1rsn7/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# Пути до датасетов с автомобилями
PATH_CAR = './DATASET/CarDetection' # Путь до датасета Car Detection
PATH_CARS = './DATASET/CarsDetection/Cars Detection/' # Путь до датасета Cars Detection
PATH_CAR_PERSON = './DATASET/CarPerson/Car-Person-v2-Roboflow-Owais-Ahmad/' # Путь до датасета Car Person

# Пути до датасетов с ЛЭП
PATH_STN = './DATASET/STN' # Путь до датасета STN PLAD
PATH_UTILLUTY_POLE = './DATASET/UtilityPole/' # Путь до датасета Utiluty Pole
PATH_TRANSMISSION = './DATASET/Transmission/data_original_size_v1/data_original_size/' # Путь до датасета Transmission

# Пути до датасетов с деревьями
PATH_TREE = './DATASET/Tree/' # Путь до датасета Tree Detection
PATH_TREE_DETECTION = './DATASET/TreeDetection/' # Путь до датасета Tree Detection
PATH_TREE_100 = './DATASET/Trees100/' # Путь до датасета Tree 100

# Пути до датасетов с птицами
PATH_BIRD_ANNOTATION = './DATASET/BirdAnnotation/' # Путь до датасета Bird Annotation 
PATH_BIRDS_DETECTOR = './DATASET/BirdsDetector/' # Путь до датасета Birds Detector
PATH_BIRDS_V3 = './DATASET/BirdsV3/' # Путь до датасета BirdsV3

# Пути до датасетов со зданиями
PATH_BUILDING_DETECTION = './DATASET/BuildingDetection/' # Путь до датасета Building Detection
PATH_JOSH_EDITS = './DATASET/JoshEdits/' # Путь до датасета JoshEdits

# 2. Обработка данных

In [3]:
def converterYoloToDataFrame(labels_list, path, classes):
    '''
        Функция позволяет перенести все данные с формата YOLO в DataFrame
        Input:
            labels_list - набор названий изображений
            path - путь до папки с изображениями
    '''
    labels = {} # Словарь объктов
    number_item = 0 # Номер записи
    for item in tqdm(labels_list):
        # Читаем все записи
        with open(path + item) as f:
            lines = f.readlines()
    
        # Проходимся по каждой записи
        for line in lines:
            # Получаем координаты в процентах
            x_center, y_center, w, h = (float(number) for number in line.split()[1:]) 
    
            # Заносим в словарь
            labels[number_item] = {
                'image_name': '/'.join(path.split('/')[:-2]) + '/images/' + item[:-3] + 'jpg',
                'x_center': x_center, 
                'y_center': y_center, 
                'w': w, 
                'h': h,
                'classes': classes
            }
            number_item += 1
    return labels

In [4]:
def datasetMinimization(dataSet, count):
    '''
        Функция позволяет понизить размер DataFrame за счёт удаления фотографий
        Input:
            dataSet - первоначальный DataFrame
            count - количество, во сколько раз сжать DataFrame
    '''
    # Список во всеми именами фотографий
    list_image_name = dataSet['image_name'].unique().tolist()

    # Кол-во фотографий в начале
    start_count_image = len(list_image_name)
    
    # Получаем именя изображений, которые оставляем
    name_file = random.sample(list_image_name, len(list_image_name) // count)

    # Записи с этими файлами
    dataSet = dataSet[dataSet['image_name'].isin(name_file)]

    # Кол-во фотографий после обработки
    end_count_image = len(dataSet['image_name'].unique())
    
    # Выведем системноо сообщение об результате сокращения фотографий
    print(f'There were {start_count_image} photos, now there are {end_count_image}')
    
    return dataSet

## 2.0 Классы для модели
    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

## 2.1. Датасеты с автомобилями
### 2.1.1 Обработка датасета Car Detection

In [5]:
img_h, img_w = (380, 676) # Размеры изображения
df = pd.read_csv(PATH_CAR + '/train_solution_bounding_boxes (1).csv') # Читаем датасет

df.rename(columns={'image':'image_name'}, inplace=True) # Переименовываем колонку

df.sample(10) # посмотрим на данные

,image_name,xmin,ymin,xmax,ymax
114,vid_4_14500.jpg,0.000000,191.925676,88.535456,229.094273
525,vid_4_9540.jpg,375.664967,173.218102,532.725419,228.839864
148,vid_4_17320.jpg,262.182344,176.764801,561.050651,308.322072
294,vid_4_2420.jpg,25.924747,193.392857,112.992764,234.962999
500,vid_4_9060.jpg,10.339403,196.352817,63.021122,223.917584
101,vid_4_13900.jpg,141.852388,182.633526,253.866860,231.050515
375,vid_4_29920.jpg,247.997106,188.013192,331.151954,227.138031
427,vid_4_6240.jpg,39.620839,195.838160,112.014472,227.627091
197,vid_4_19040.jpg,548.332851,178.721043,659.369030,227.627091
480,vid_4_8300.jpg,124.565186,192.907221,225.005098,234.746600


In [6]:
# Получим координаты центра объекта в процентах 
df['x_center'] = (df['xmin'] + df['xmax']) / 2 / img_w 
df['y_center'] = (df['ymin'] + df['ymax']) / 2 / img_h 

# Получим размеры объекта в процентах
df['w'] = (df['xmax'] - df['xmin']) / img_w # Получаем ширину объекта в процентах
df['h'] = (df['ymax'] - df['ymin']) / img_h # Получаем ширину объекта в процентах
df['classes'] = 0 # Задаём класс объекта
df.drop(
    ['xmin', 'xmax', 'ymin', 'ymax'], # Удаляем ненужные признаки
    axis=1, inplace=True
)

# Изменим путь до файла
df['image_name'] = f'{PATH_CAR}/training_images/' + df['image_name']

# Очистим память 
del img_h, img_w

# Уменьшаем размер DataFrame
df = datasetMinimization(df, 2)

df.sample(10) # Посмотрим на данные

There were 355 photos, now there are 177


,image_name,x_center,y_center,w,h,classes
192,./DATASET/CarDetection/training_images/vid_4_1...,0.532200,0.549469,0.210564,0.140283,0
158,./DATASET/CarDetection/training_images/vid_4_1...,0.506512,0.573279,0.270622,0.180180,0
406,./DATASET/CarDetection/training_images/vid_4_3...,0.433792,0.568774,0.168596,0.119691,0
383,./DATASET/CarDetection/training_images/vid_4_2...,0.376990,0.555904,0.096961,0.068211,0
221,./DATASET/CarDetection/training_images/vid_4_2...,0.206946,0.579714,0.160637,0.110682,0
3,./DATASET/CarDetection/training_images/vid_4_1...,0.833213,0.531451,0.197540,0.155727,0
241,./DATASET/CarDetection/training_images/vid_4_2...,0.039436,0.550113,0.078871,0.100386,0
442,./DATASET/CarDetection/training_images/vid_4_6...,0.132779,0.562339,0.152677,0.106821,0
405,./DATASET/CarDetection/training_images/vid_4_3...,0.919320,0.528234,0.161360,0.115830,0
316,./DATASET/CarDetection/training_images/vid_4_2...,0.831766,0.527590,0.185962,0.101673,0


## 2.1.2. Обработка датасета Cars Detection

In [7]:
dataCarsDetection = pd.DataFrame(columns=df.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CARS}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CARS}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarsDetection = pd.concat([dataCarsDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarsDetection = datasetMinimization(dataCarsDetection, 3)

dataCarsDetection.sample(10) # Посмотрим на данные

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

There were 1254 photos, now there are 418


,image_name,x_center,y_center,w,h,classes
109,./DATASET/CarsDetection/Cars Detection/test/im...,0.38101,0.423077,0.358173,0.453125,0
171,./DATASET/CarsDetection/Cars Detection/valid/i...,0.467548,0.371394,0.049279,0.066106,0
1173,./DATASET/CarsDetection/Cars Detection/train/i...,0.870192,0.530048,0.258413,0.13101,0
480,./DATASET/CarsDetection/Cars Detection/train/i...,0.175481,0.825721,0.19351,0.115385,0
10,./DATASET/CarsDetection/Cars Detection/valid/i...,0.734375,0.50601,0.527644,0.328125,0
311,./DATASET/CarsDetection/Cars Detection/valid/i...,0.144231,0.348558,0.169471,0.114183,0
590,./DATASET/CarsDetection/Cars Detection/train/i...,0.421875,0.528846,0.509615,0.710337,0
821,./DATASET/CarsDetection/Cars Detection/train/i...,0.762019,0.510817,0.471154,0.420673,0
942,./DATASET/CarsDetection/Cars Detection/train/i...,0.137019,0.383413,0.270433,0.135817,0
105,./DATASET/CarsDetection/Cars Detection/valid/i...,0.385817,0.554087,0.771635,0.887019,0


### 2.1.3. Обработка датасета Car Person

In [8]:
dataCarPerson = pd.DataFrame(columns=df.columns)
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CAR_PERSON}{sample}/labels') # Список файлов с разметкой
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CAR_PERSON}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarPerson = pd.concat([dataCarPerson, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarPerson = datasetMinimization(dataCarPerson, 4)

dataCarPerson.sample(10) # Посмотрим на данные

  0%|          | 0/1571 [00:00<?, ?it/s]

  0%|          | 0/448 [00:00<?, ?it/s]

  0%|          | 0/224 [00:00<?, ?it/s]

There were 2243 photos, now there are 560


,image_name,x_center,y_center,w,h,classes
11198,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.163462,0.759615,0.112981,0.137019,0
7773,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.819712,0.778846,0.328125,0.106971,0
1170,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.254808,0.415865,0.036058,0.120192,0
10376,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.706731,0.646635,0.028846,0.121394,0
2357,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.151442,0.560096,0.033654,0.091346,0
3391,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.300481,0.384615,0.02524,0.063702,0
2138,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.066106,0.25,0.111779,0.122596,0
8695,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.111779,0.591346,0.223558,0.778846,0
4609,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.388221,0.682692,0.221154,0.543269,0
8987,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.317308,0.473558,0.024038,0.174279,0


## 2.2. Обработка датасет с ЛЭП
### 2.2.1 Обработка датасета STN_PLAD

In [9]:
# Читаем файл с разметкой датасета STN
with open(PATH_STN + '/annotations.json') as f:
    dataSTN = json.load(f)

pprint(dataSTN['categories']) # Посмотрим на категории

# Получим два DataFrame, один с изображениями, другой с разметкой объектов
data = pd.DataFrame(dataSTN['images']).rename(columns={'id': 'image_id'})
data1 =  pd.DataFrame(dataSTN['annotations'])

# Объединим два дасатсета и удалим лишние признаки
dataSTN = pd.merge(data, data1, on='image_id', how='left').drop(
    [
        'licence', 'segmentation', 'date_captured', 'coco_url', 
        'iscrowd', 'image_id'
    ], axis=1
)

# Очистим память 
del data, data1 

dataSTN.sample(10) # Посмотрим на данные

[{'id': 0, 'name': 'tower', 'supercategory': 'tower'},
 {'id': 1, 'name': 'insulator', 'supercategory': 'component'},
 {'id': 2, 'name': 'spacer', 'supercategory': 'component'},
 {'id': 3, 'name': 'damper', 'supercategory': 'component'},
 {'id': 5, 'name': 'plate', 'supercategory': 'component'}]


,file_name,height,width,flickr_url,area,bbox,category_id,id
2106,20181129-R/DJI_0011.JPG,3648,5472,,27216,"[1196, 1336, 63, 432]",1,2117
361,20181127-A1/DJI_0440.JPG,3078,5472,,1748,"[2290, 1848, 76, 23]",3,362
1484,20181127-A1/DJI_0465.JPG,3078,5472,,1472,"[2660, 1833, 32, 46]",3,1491
1295,20181127-A1/DJI_0451.JPG,3078,5472,,1972,"[2899, 2224, 34, 58]",3,1302
1696,20181127-A1/DJI_0489.JPG,3078,5472,,8468,"[2333, 1706, 58, 146]",2,1704
723,20181127-A1/DJI_0493.JPG,3078,5472,,20787,"[2879, 2204, 169, 123]",2,726
1713,20181127-A1/DJI_0419.JPG,3078,5472,,71451,"[2683, 1154, 153, 467]",1,1721
1340,20181129-R/DJI_0025.JPG,3648,5472,,6438,"[166, 3233, 58, 111]",3,1347
434,20181129-R/DJI_0020.JPG,3648,5472,,1992,"[2568, 1968, 83, 24]",3,435
279,20181127-A1/DJI_0455.JPG,3078,5472,,1298,"[2769, 1936, 22, 59]",3,280


In [10]:
# Получим координаты объекта
dataSTN[['xmin', 'ymin', 'xmax', 'ymax']] = dataSTN['bbox'].apply(lambda x: pd.Series(x))
dataSTN['xmax'] = dataSTN['xmax'] + dataSTN['xmin']
dataSTN['ymax'] = dataSTN['ymax'] + dataSTN['ymin']

# Получим координаты центра объекта в процентах 
dataSTN['x_center'] = (dataSTN['xmin'] + dataSTN['xmax']) / 2 / dataSTN['width']
dataSTN['y_center'] = (dataSTN['ymin'] + dataSTN['ymax']) / 2 / dataSTN['height']

# Получим размеры объекта в процентах
dataSTN['w'] = (dataSTN['xmax'] - dataSTN['xmin']) / dataSTN['width']
dataSTN['h'] = (dataSTN['ymax'] - dataSTN['ymin']) / dataSTN['height']

# Переименуем признаки
dataSTN.rename(
    columns={
        'file_name': 'image_name',
        'category_id': 'classes'
    }, 
    inplace=True
) 

# Удалим ненужные признаки
dataSTN.drop(
    [
        'height', 'width', 'flickr_url',
        'area', 'bbox', 'id', 'xmin', 
        'xmax', 'ymin', 'ymax'
    ], 
    axis=1, inplace=True
)

# Изменим путь до файла
dataSTN['image_name'] = './DATASET/STN/' + dataSTN['image_name']

# Оставим фотографии принадлижащие только к опоре ЛЭП
dataSTN = dataSTN[dataSTN['classes'] == 0]
dataSTN['classes'] = dataSTN['classes'] + 1

dataSTN.sample(10) # Посомтрим на данные

,image_name,classes,x_center,y_center,w,h
1505,./DATASET/STN/20181127-A1/DJI_0490.JPG,1,0.786550,0.087232,0.100146,0.173814
1285,./DATASET/STN/20181127-A1/DJI_0430.JPG,1,0.213085,0.331059,0.111111,0.450942
1552,./DATASET/STN/20181127-A1/DJI_0494.JPG,1,0.401133,0.500162,0.264254,0.999675
2221,./DATASET/STN/20181127-A1/DJI_0466.JPG,1,0.718750,0.659357,0.562500,0.681287
1507,./DATASET/STN/20181127-A1/DJI_0490.JPG,1,0.467014,0.546134,0.134320,0.907732
1103,./DATASET/STN/20181129-R/DJI_0016.JPG,1,0.776590,0.427906,0.015168,0.098136
1196,./DATASET/STN/20181127-A1/DJI_0503.JPG,1,0.514072,0.636615,0.648757,0.726771
1508,./DATASET/STN/20181127-A1/DJI_0428.JPG,1,0.515716,0.500162,0.119883,0.999675
196,./DATASET/STN/20181127-A1/DJI_0414.JPG,1,0.291667,0.269331,0.092836,0.421053
2400,./DATASET/STN/20181127-A1/DJI_0514.JPG,1,0.283991,0.682099,0.545322,0.635802


### 2.2.2. Обработка датасета Utility Pole

In [11]:
dataUnilityPole = pd.DataFrame(columns=df.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'test']:
    labels_name = os.listdir(f'{PATH_UTILLUTY_POLE}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_UTILLUTY_POLE}{sample}/labels/', 1)) # Получаем датасет с разметкой
    dataUnilityPole = pd.concat([dataUnilityPole, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память
dataUnilityPole.sample(10) # Посмотрим на данные

  0%|          | 0/179 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
24,./DATASET/UtilityPole/train/images/images--5-_...,0.354567,0.536058,0.247596,0.8125,1
170,./DATASET/UtilityPole/train/images/images--5-_...,0.645433,0.536058,0.247596,0.8125,1
22,./DATASET/UtilityPole/train/images/images--8-_...,0.473558,0.449519,0.221154,0.899038,1
4,./DATASET/UtilityPole/train/images/a93a35ee792...,0.853365,0.519231,0.283654,0.778846,1
89,./DATASET/UtilityPole/train/images/images--2-_...,0.36899,0.394231,0.362981,0.677885,1
147,./DATASET/UtilityPole/train/images/images--3-_...,0.259615,0.5,0.298077,0.995192,1
93,./DATASET/UtilityPole/train/images/distributio...,0.496394,0.512019,0.165865,0.682692,1
45,./DATASET/UtilityPole/train/images/Utility_Pol...,0.677885,0.491587,0.197115,0.752404,1
197,./DATASET/UtilityPole/train/images/774-utility...,0.341346,0.543269,0.0625,0.264423,1
189,./DATASET/UtilityPole/train/images/istockphoto...,0.63101,0.705529,0.127404,0.396635,1


### 2.2.3. Обработка датасета Transmission

In [12]:
list_image = glob.glob(PATH_TRANSMISSION + '*.jpg')
labels = {} # Словарь объктов
number_item = 0 # Номер записи
for path_image in tqdm(list_image):
    with open(path_image[:-3] + 'json') as f:
        json_file = json.load(f)
    xmax = 0
    xmin = 2**10
    ymax = 0
    ymin = 2**10
    height = json_file['imageHeight']
    width = json_file['imageWidth']
    for item in json_file['shapes']:
        if item['label'] in ['tower_wooden', 'cable']:
            for data in item['points']:
                x, y = data
                xmax = x if x > xmax else xmax
                ymax = y if x > ymax else ymax
                xmin = x if y < xmin else xmin
                ymin = y if y < ymin else ymin
            x_center = (xmin + xmax) / 2 / width
            y_center = (ymin + ymax) / 2 / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height
            labels[number_item] = {
                'image_name': path_image,
                'x_center': 0.0 if x_center < 0.0 else 1.0 if x_center > 1.0 else x_center, 
                'y_center': 0.0 if y_center < 0.0 else 1.0 if y_center > 1.0 else x_center, 
                'w': 0.0 if w < 0.0 else 1.0 if w > 1.0 else w, 
                'h': 0.0 if h < 0.0 else 1.0 if h > 1.0 else h,
                'classes': 1 if item['label'] == 'tower_wooden' else 5
            }
            number_item += 1
            
dataTransmission = pd.DataFrame(labels).T # Переносим все в DataFrame

# Очищаем память
del list_image, labels, number_item, xmin, xmax, ymin, ymax, h, w, json_file 

# Уменьшаем размер DataFrame
dataTransmission = datasetMinimization(dataTransmission, 2)

dataTransmission.sample(10) # Посмотрим на данные

  0%|          | 0/1242 [00:00<?, ?it/s]

There were 1231 photos, now there are 615


,image_name,x_center,y_center,w,h,classes
5138,./DATASET/Transmission/data_original_size_v1/d...,0.482407,0.482407,0.683624,0.999191,5
883,./DATASET/Transmission/data_original_size_v1/d...,0.754694,0.754694,0.490091,0.0,5
9634,./DATASET/Transmission/data_original_size_v1/d...,0.36067,0.36067,0.720898,0.748985,5
2216,./DATASET/Transmission/data_original_size_v1/d...,0.722606,0.722606,0.04958,0.041712,5
7686,./DATASET/Transmission/data_original_size_v1/d...,0.472824,0.472824,0.353879,0.924506,5
9542,./DATASET/Transmission/data_original_size_v1/d...,0.500147,0.500147,0.999186,0.325047,5
1512,./DATASET/Transmission/data_original_size_v1/d...,0.427474,0.427474,0.854948,0.951075,5
2206,./DATASET/Transmission/data_original_size_v1/d...,0.723339,0.723339,0.048114,0.643024,5
7625,./DATASET/Transmission/data_original_size_v1/d...,0.879557,0.879557,0.222656,0.999537,5
4848,./DATASET/Transmission/data_original_size_v1/d...,0.659884,0.659884,0.218508,0.03876,5


## 2.3. Обработка датасетов с деревьями
### 2.3.1. Обработка датасета Tree Object Detecion

In [13]:
dataTree = pd.DataFrame(columns=df.columns)
labels_name = os.listdir(f'{PATH_TREE}train/labels/') # Список файлов с разметкой
labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE}train/labels/', 2)) # Получаем датасет с разметкой
dataTree = pd.concat([dataTree, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

dataTree.sample(10) # Посмотрим на данные

  0%|          | 0/214 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
42,./DATASET/Tree/train/images/trees-isolated-172...,0.290625,0.75625,0.509375,0.417187,2
45,./DATASET/Tree/train/images/sacramento_streett...,0.3375,0.466406,0.3125,0.4625,2
680,./DATASET/Tree/train/images/pexels-adrien-olic...,0.704688,0.510938,0.582812,0.871875,2
230,./DATASET/Tree/train/images/landscape-5591270_...,0.366406,0.421875,0.628125,0.696875,2
668,./DATASET/Tree/train/images/20230709_155406_jp...,0.528125,0.446875,0.55625,0.89375,2
216,./DATASET/Tree/train/images/TCC-2021_jpg.rf.b5...,0.102344,0.109375,0.198437,0.21875,2
604,./DATASET/Tree/train/images/images-20-_jpg.rf....,0.04375,0.314063,0.06875,0.315625,2
321,./DATASET/Tree/train/images/big-tree-pixabay_j...,0.647656,0.954688,0.085938,0.090625,2
371,./DATASET/Tree/train/images/20230709_155608_jp...,0.183594,0.325781,0.365625,0.614062,2
474,./DATASET/Tree/train/images/istockphoto-117402...,0.099219,0.248438,0.198437,0.482812,2


### 2.3.2. Обработка датасета TreeDetection 

In [14]:
dataTreeDetection = pd.DataFrame(columns=df.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_DETECTION}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTreeDetection = pd.concat([dataTreeDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataTreeDetection = datasetMinimization(dataTreeDetection, 2)

dataTreeDetection.sample(10) # Посмотрим на данные

  0%|          | 0/1803 [00:00<?, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

There were 2060 photos, now there are 1030


,image_name,x_center,y_center,w,h,classes
865,./DATASET/TreeDetection/train/images/93e728fa-...,0.299219,0.404687,0.06875,0.425,2
2119,./DATASET/TreeDetection/train/images/f3b40d17-...,0.257031,0.341406,0.278125,0.68125,2
296,./DATASET/TreeDetection/valid/images/1176250b-...,0.452344,0.483594,0.528906,0.780469,2
2747,./DATASET/TreeDetection/train/images/230804_17...,0.192188,0.558594,0.292969,0.5125,2
1487,./DATASET/TreeDetection/train/images/daf79323-...,0.4,0.496875,0.55,0.915625,2
909,./DATASET/TreeDetection/train/images/221011_17...,0.33125,0.478125,0.625,0.817969,2
162,./DATASET/TreeDetection/train/images/211012_17...,0.351562,0.789844,0.703125,0.420312,2
376,./DATASET/TreeDetection/train/images/6cacf84a-...,0.510156,0.492969,0.947656,0.878906,2
689,./DATASET/TreeDetection/train/images/221015_17...,0.413281,0.303125,0.56875,0.25,2
995,./DATASET/TreeDetection/train/images/211009_13...,0.053906,0.578906,0.10625,0.50625,2


### 2.3.3. Обработка датасета Tree 100

In [15]:
dataTree100 = pd.DataFrame(columns=df.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_100}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_100}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTree100 = pd.concat([dataTree100, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

dataTree100.sample(10) # Посмотрим на данные

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
37,./DATASET/Trees100/train/images/arvore115_jpg....,0.464844,0.786719,0.0875,0.308594,2
10,./DATASET/Trees100/train/images/arvore29_jpg.r...,0.522656,0.728906,0.321875,0.342187,2
132,./DATASET/Trees100/train/images/arvore51_jpg.r...,0.541406,0.382031,0.180469,0.265625,2
14,./DATASET/Trees100/train/images/arvore89_jpg.r...,0.4875,0.532813,0.600781,0.801562,2
138,./DATASET/Trees100/train/images/arvore119_jpg....,0.521875,0.607031,0.584375,0.785937,2
73,./DATASET/Trees100/train/images/arvore82_jpg.r...,0.5625,0.348438,0.1,0.239063,2
136,./DATASET/Trees100/train/images/arvore104_jpg....,0.372656,0.547656,0.669531,0.904687,2
44,./DATASET/Trees100/train/images/arvore49_jpg.r...,0.141406,0.589844,0.069531,0.135937,2
31,./DATASET/Trees100/valid/images/arvore42_jpg.r...,0.778125,0.35625,0.071875,0.207813,2
114,./DATASET/Trees100/train/images/arvore40_jpg.r...,0.775,0.720313,0.025781,0.046875,2


## 2.4. Обработка датасета с птицами
### 2.4.1. Обработка датасета с Bird Annotation

In [16]:
dataBirdAnnotation = pd.DataFrame(columns=df.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRD_ANNOTATION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRD_ANNOTATION}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdAnnotation  = pd.concat([dataBirdAnnotation, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdAnnotation = datasetMinimization(dataBirdAnnotation, 2)


dataBirdAnnotation.sample(10) # Посмотрим на данные

  0%|          | 0/1362 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

There were 1557 photos, now there are 778


,image_name,x_center,y_center,w,h,classes
841,./DATASET/BirdAnnotation/valid/images/voladora...,0.251563,0.490625,0.03125,0.025,3
756,./DATASET/BirdAnnotation/valid/images/d54309c2...,0.875,0.517188,0.020313,0.010156,3
854,./DATASET/BirdAnnotation/valid/images/voladora...,0.491406,0.211719,0.029687,0.014063,3
18835,./DATASET/BirdAnnotation/train/images/voladora...,0.407813,0.01875,0.035937,0.0125,3
16175,./DATASET/BirdAnnotation/train/images/e129_jpg...,0.888281,0.622656,0.029687,0.060937,3
14702,./DATASET/BirdAnnotation/train/images/voladora...,0.671094,0.474219,0.071875,0.048438,3
12616,./DATASET/BirdAnnotation/train/images/e179_jpg...,0.55,0.39375,0.021875,0.040625,3
333,./DATASET/BirdAnnotation/test/images/voladoras...,0.799219,0.504687,0.064062,0.046875,3
3187,./DATASET/BirdAnnotation/train/images/pepa65_j...,0.258594,0.421094,0.48125,0.478125,3
12720,./DATASET/BirdAnnotation/train/images/gg14_jpg...,0.585938,0.225781,0.021875,0.023438,3


### 2.4.2. Обработка датасета с Birds Detection

In [17]:
dataBirdsDetectod = pd.DataFrame(columns=df.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_DETECTOR}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_DETECTOR}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsDetectod = pd.concat([dataBirdsDetectod, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsDetectod = datasetMinimization(dataBirdsDetectod, 2)

dataBirdsDetectod.sample(10)  # Посмотрим на данные

  0%|          | 0/594 [00:00<?, ?it/s]

  0%|          | 0/170 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

There were 849 photos, now there are 424


,image_name,x_center,y_center,w,h,classes
2082,./DATASET/BirdsDetector/train/images/voladoras...,0.166406,0.789062,0.160156,0.154688,3
3083,./DATASET/BirdsDetector/train/images/voladoras...,0.689063,0.160938,0.182031,0.257031,3
910,./DATASET/BirdsDetector/train/images/e138_jpg....,0.217969,0.194531,0.105469,0.107031,3
4012,./DATASET/BirdsDetector/train/images/voladoras...,0.752344,0.784375,0.105469,0.076563,3
1407,./DATASET/BirdsDetector/train/images/voladoras...,0.375781,0.710938,0.066406,0.066406,3
119,./DATASET/BirdsDetector/test/images/voladorasa...,0.3,0.410938,0.142187,0.110937,3
2763,./DATASET/BirdsDetector/train/images/descarga-...,0.109375,0.683594,0.041406,0.103125,3
1879,./DATASET/BirdsDetector/valid/images/jisus-19-...,0.642969,0.667969,0.423438,0.442969,3
2741,./DATASET/BirdsDetector/train/images/descarga-...,0.602344,0.732812,0.048438,0.103125,3
1457,./DATASET/BirdsDetector/train/images/gg13_jpg....,0.382812,0.753906,0.15625,0.139844,3


### 2.4.3. Обработка датасета с BirdsV3

In [18]:
dataBirdsV3 = pd.DataFrame(columns=df.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_V3}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_V3}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsV3 = pd.concat([dataBirdsV3, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsV3 = datasetMinimization(dataBirdsV3, 2)

dataBirdsV3.sample(10)  # Посмотрим на данные

  0%|          | 0/582 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

There were 665 photos, now there are 332


,image_name,x_center,y_center,w,h,classes
2946,./DATASET/BirdsV3/train/images/birds-roof-8390...,0.959375,0.373437,0.051562,0.060937,3
236,./DATASET/BirdsV3/valid/images/Roofing-Issues-...,0.713281,0.621094,0.108594,0.152344,3
135,./DATASET/BirdsV3/valid/images/pigeons-roof-th...,0.671094,0.296094,0.100781,0.057813,3
1947,./DATASET/BirdsV3/train/images/pigeonroofdamag...,0.490625,0.222656,0.059375,0.084375,3
4115,./DATASET/BirdsV3/train/images/83ff5a3360c7b39...,0.314063,0.563281,0.05,0.042188,3
3951,./DATASET/BirdsV3/train/images/pigeons-roof-th...,0.530469,0.283594,0.0625,0.096875,3
132,./DATASET/BirdsV3/valid/images/pigeons-roof-th...,0.496094,0.309375,0.045312,0.095312,3
228,./DATASET/BirdsV3/train/images/4541968436_353e...,0.689063,0.340625,0.05,0.05625,3
1108,./DATASET/BirdsV3/train/images/high-angle-floc...,0.578125,0.2625,0.045312,0.057813,3
2823,./DATASET/BirdsV3/train/images/photo-156466976...,0.413281,0.575,0.073438,0.064062,3


## 2.5. Обработка фотографий со зданиями
### 2.5.1. Обработка датасета Building Detection 

In [19]:
dataBuildingDetection = pd.DataFrame(columns=df.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BUILDING_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BUILDING_DETECTION}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataBuildingDetection = pd.concat([dataBuildingDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBuildingDetection = datasetMinimization(dataBuildingDetection, 2)

dataBuildingDetection.sample(10)  # Посмотрим на данные

  0%|          | 0/726 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

There were 802 photos, now there are 401


,image_name,x_center,y_center,w,h,classes
497,./DATASET/BuildingDetection/train/images/28_A_...,0.5,0.498798,1.0,0.997596,4
635,./DATASET/BuildingDetection/train/images/400_A...,0.498798,0.5,0.997596,1.0,4
247,./DATASET/BuildingDetection/train/images/189_A...,0.498798,0.498798,0.997596,0.997596,4
294,./DATASET/BuildingDetection/train/images/192_A...,0.498798,0.498798,0.997596,0.997596,4
359,./DATASET/BuildingDetection/train/images/3_A_j...,0.52524,0.526442,0.911058,0.942308,4
504,./DATASET/BuildingDetection/train/images/300_A...,0.498798,0.498798,0.997596,0.997596,4
432,./DATASET/BuildingDetection/train/images/183_A...,0.428019,0.49664,0.843197,0.993279,4
12,./DATASET/BuildingDetection/test/images/148_A_...,0.497596,0.606971,0.668269,0.777644,4
362,./DATASET/BuildingDetection/train/images/29_A_...,0.5,0.498798,1.0,0.997596,4
272,./DATASET/BuildingDetection/train/images/116_A...,0.728876,0.544703,0.373143,0.686664,4


### 2.5.2. Обработка датасета Josh Edits 

In [20]:
dataJoshEdits = pd.DataFrame(columns=df.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_JOSH_EDITS}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_JOSH_EDITS}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataJoshEdits = pd.concat([dataJoshEdits, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataJoshEdits = datasetMinimization(dataJoshEdits, 3)

dataJoshEdits.sample(10)  # Посмотрим на данные

  0%|          | 0/3042 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

There were 3257 photos, now there are 1085


,image_name,x_center,y_center,w,h,classes
1765,./DATASET/JoshEdits/train/images/sv_37-4114645...,0.109375,0.274219,0.21875,0.346094,4
4638,./DATASET/JoshEdits/train/images/image_109_png...,0.914062,0.676562,0.171875,0.4375,4
1941,./DATASET/JoshEdits/train/images/streetview_10...,0.375781,0.55,0.4625,0.4,4
3717,./DATASET/JoshEdits/train/images/debug_image_2...,0.035156,0.153906,0.070312,0.110156,4
2711,./DATASET/JoshEdits/train/images/b1647b6b5f299...,0.5,0.5,1.0,1.0,4
692,./DATASET/JoshEdits/train/images/download_jpg....,0.5,0.5,1.0,1.0,4
488,./DATASET/JoshEdits/valid/images/sv_42-2510104...,0.669531,0.486719,0.322656,0.1375,4
1296,./DATASET/JoshEdits/train/images/sv_42-3553559...,0.555469,0.476562,0.214062,0.220312,4
1927,./DATASET/JoshEdits/train/images/sv_42-1123531...,0.648438,0.467187,0.090625,0.03125,4
4132,./DATASET/JoshEdits/train/images/1520374445560...,0.5,0.364844,1.0,0.729688,4


# 3. Объединение датасетов в единый датасет

In [21]:
data = pd.concat(
    [
        df, dataCarPerson, dataCarsDetection, # Авто
        dataSTN, dataUnilityPole, dataTransmission, # ЛЭП
        dataTree, dataTreeDetection, dataTree100, # Деревья
        dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3, # Птицы
        dataBuildingDetection, dataJoshEdits # Здания
    ], 
    ignore_index=True
) # Объеденим датасеты в один единый

del df, dataCarPerson, dataCarsDetection # очищаем память от датасетов с Авто
del dataSTN, dataUnilityPole, dataTransmission # очищаем память от датасетов с ЛЭП
del dataTree, dataTreeDetection, dataTree100 # очищаем память от датасетов со Деревьями
del dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3 # очищаем память от датасетов со Птицами
del dataBuildingDetection, dataJoshEdits # очищаем память от датасетов со Зданиями

data.sample(10) # Посмотрим на данные

,image_name,x_center,y_center,w,h,classes
24401,./DATASET/BirdAnnotation/valid/images/voladora...,0.703906,0.459375,0.020313,0.03125,3
10427,./DATASET/Transmission/data_original_size_v1/d...,0.422396,0.422396,0.0,0.0,5
21433,./DATASET/BirdAnnotation/train/images/pajarosc...,0.719531,0.246094,0.059375,0.085938,3
2614,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.707933,0.367788,0.055288,0.338942,0
32750,./DATASET/JoshEdits/train/images/sv_42-2602140...,0.675,0.503906,0.253125,0.123438,4
6504,./DATASET/Transmission/data_original_size_v1/d...,0.49987,0.49987,0.99974,0.265365,5
30651,./DATASET/BirdsV3/valid/images/24123101625_ef5...,0.214844,0.525,0.054688,0.074219,3
9820,./DATASET/Transmission/data_original_size_v1/d...,0.53515,0.53515,0.92918,0.546762,5
12845,./DATASET/TreeDetection/train/images/230930_17...,0.410938,0.814063,0.734375,0.346875,2
24244,./DATASET/BirdAnnotation/valid/images/voladora...,0.146875,0.374219,0.04375,0.023438,3


In [22]:
data['image_name'].unique().shape # Количество фотографий

(6468,)

In [23]:
# Изменим тип данных координат с string на float
for column in data.columns[1:-1]:
    data[column] = data[column].astype(float)

data['classes'] = data['classes'].astype(int) # Изменим тип классов с string на int
data.info() # Посмотрим на сведение дата сета

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33399 entries, 0 to 33398
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  33399 non-null  object 
 1   x_center    33399 non-null  float64
 2   y_center    33399 non-null  float64
 3   w           33399 non-null  float64
 4   h           33399 non-null  float64
 5   classes     33399 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 1.5+ MB


In [24]:
count_image = data['image_name'].unique().shape[0]
print(f'Dataset contains {count_image} unique photos and {data.shape[0]} objects')
del count_image # Очищаем память

Dataset contains 6468 unique photos and 33399 objects


    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [25]:
# Выведем в процентах количество уникальных изображений по классам
data.groupby('image_name')['classes'].head().value_counts().sort_index() /  data.groupby('image_name')['classes'].head().shape[0] * 100

classes
0    20.646364
1     4.008701
2    15.798633
3    28.713487
4    16.152890
5    14.679925
Name: count, dtype: float64

    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [26]:
# Распределение классов среди всех объектов
data['classes'].value_counts().sort_index() / data.shape[0] * 100

classes
0    15.754963
1     2.179706
2     8.305638
3    50.552412
4     7.844546
5    15.362735
Name: count, dtype: float64

In [27]:
# Разделим данные на выборки
train_name_file, test_name_file = train_test_split(data['image_name'].unique(), test_size=.30, random_state=42) 
valid_name_file, test_name_file = train_test_split(test_name_file, test_size=.33, random_state=42)

# Посмотрим на процентное отношение выборок 
pd.DataFrame([len(train_name_file), len(valid_name_file), len(test_name_file)], columns=['size']) / len(data['image_name'].unique()) * 100

,size
0,69.990724
1,20.098949
2,9.910328


In [28]:
train_df = data[data['image_name'].isin(train_name_file)] # Записи для тренировочных данных
valid_df = data[data['image_name'].isin(valid_name_file)] # Записи для валидационных данных
test_df = data[data['image_name'].isin(test_name_file)] # Записи для тестовых данных
del data, train_name_file, valid_name_file, test_name_file # Очистим память

In [29]:
!mkdir DATA # Создаём папку с исходным датасетов

!mkdir DATA/train # Создаём папку с тренировочной выборкой 
!mkdir DATA/valid # Создаём папку с валидационной выборкой 
!mkdir DATA/test # Создаём папку с тестовой выборкой 

!mkdir DATA/train/images # Создаём папку с изображениями для тренировочной выборки
!mkdir DATA/train/labels # Создаём папку с разметкой для тренировочной выборки
!mkdir DATA/valid/images # Создаём папку с изображениями для валидационной выборки
!mkdir DATA/valid/labels # Создаём папку с разметкой для валидационной выборки
!mkdir DATA/test/images # Создаём папку с изображениями для тестовой выборки
!mkdir DATA/test/labels # Создаём папку с разметкой для тестовой выборки

In [30]:
def copyFileDataSet(df, pathEnd):
    '''
        Функция позволяет скопировать фото в новую папку и под каждую из них создать txt файл с разметкой
        Input:
            df - DataFrame содержащий в себе следующиго типа:
                Признаки    Non-Null?  Тип
                image_name  non-null   object 
                x_center    non-null   float64
                y_center    non-null   float64
                w           non-null   float64
                h           non-null   float64
                classes     non-null   int64
            pathEnd - путь созданной папки, внутри которой созданы папки
                images и labels
    '''
    nameFiles = df['image_name'].unique() # Получаем список уникальных файлов

    # Запускаем цикл по все файлам
    for item in tqdm(nameFiles):

        # shutil.copy2(item, pathEnd+'/images') # Копируем данный файл в другую директорию/
        img = cv2.imread(item, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(cv2.UMat(img), (1280, 720), cv2.INTER_NEAREST)
        cv2.imwrite(pathEnd + '/images/' + item.split('/')[-1], img)
        
        dataList = list(df[df['image_name'] == item].values) # Получаем все объекты на фото
        labelsFile = open(pathEnd + '/labels/' + item.split('/')[-1][:-3] + 'txt', 'w+') # Создаём текстовый документ с разметкой
        isFirst = True # Флаг первой строки

        # Запускаем цикл по всем объектам на фото
        for data in dataList:

            # Получаем линию с данными 
            line = '' if isFirst else '\n'
            isFirst = False
            line += str(data[-1]) + ' ' + ' '.join(str(number) for number in data[1:-1]) 
            
            labelsFile.write(line) # Записываем в файл
        labelsFile.close() # Закрываем файл
        
copyFileDataSet(train_df, './DATA/train') # Формируем тренировочную выборку
copyFileDataSet(valid_df, './DATA/valid') # Формируем валидационную выборку
copyFileDataSet(test_df, './DATA/test') # Формируем тестовую выборку
del train_df, valid_df, test_df # Очищаем память

  0%|          | 0/4527 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/641 [00:00<?, ?it/s]

In [5]:
# Глобальный путь до директории с проектом 
global_path = os.path.abspath('.') 
yolo_format = dict(
    path=f'{global_path}/DATA',
    train='../train',
    val='../valid',
    test='../test',       
    nc=6,
    names={
        0: 'Car',
        1: 'STN',
        2: 'Tree',
        3: 'Bird',
        4: 'House',
        5: 'Cable',
    }
)

del global_path # Очищаем память
             
with open('./DATA/yolo.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [6]:
epochTraning = 1 # Количество эпох обучения
pathYalm = "./DATA/yolo.yaml" # Путь до файла yalm
patience = 10 # Терпимость эпох для переобучения
# imgsz = 1280 # Размер изображения

In [7]:
model = YOLO('yolov8n.yaml').load('yolov8n.pt')
model.train(data=pathYalm, epochs=epochTraning, patience=patience)  

Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.1.24 🚀 Python-3.10.13 torch-2.2.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3904MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=./DATA/yolo.yaml, epochs=1, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_co

train: Scanning /home/k1rsn7/K1rsN7/Notebook/ВШЭ 202

train: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/train/images/07_1875.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/train/images/09_9180.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/train/images/1000_00539.jpg: 7 duplicate labels removed
train: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/train/images/104_2190.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/train/images/105_6705.jpg: 4 duplicate labels removed
train: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/train/images/106_135.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/train/images/110_105.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/train/images/111_3600.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/train/images/111_6240


val: Scanning /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/

val: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/valid/images/106_1395.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/valid/images/111_3585.jpg: 4 duplicate labels removed
val: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/valid/images/25_00475.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/valid/images/26_75.jpg: 10 duplicate labels removed
val: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/valid/images/31_00264.jpg: 4 duplicate labels removed
val: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/valid/images/34_1140.jpg: 4 duplicate labels removed
val: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/valid/images/34_1395.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/valid/images/37_00879.jpg: 10 duplicate labels removed
val: WARNING ⚠️ /home/k1rsn7/K1rsN7/Notebook/ВШЭ 2024/DATA/valid/images/39_00161.jpg: 1 duplicate l

...

        1/1      3.09G      1.632      2.444      1.
                 Class     Images  Instances      Bo


                   all       1300       7142      0.485      0.402       0.39      0.203

1 epochs completed in 0.028 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.2MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.1.24 🚀 Python-3.10.13 torch-2.2.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3904MiB)
YOLOv8n summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Bo


                   all       1300       7142      0.486      0.402       0.39      0.203
                   Car       1300       1224      0.655      0.314      0.387        0.2
                   STN       1300        149       0.49       0.49      0.422      0.221
                  Tree       1300        553      0.489      0.571      0.499      0.262
                  Bird       1300       3896       0.52      0.474      0.442      0.172
                 House       1300        526      0.658       0.51      0.571      0.357
                 Cable       1300        794      0.101     0.0529     0.0179     0.0048
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x70a4b85cfc40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     